In [1]:
import tensorflow as tf
import numpy as np

/Users/shubhan/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


##### mnist data contains images of digit 0-9 so 10 classes are there to predict

###### we are going to have three fully connected hidden layers  each layer will have 500 neurons

In [3]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500
n_classes = 10   
batch_size = 128

In [4]:
print(mnist.train.num_examples)
print(mnist.test.num_examples)

55000
10000


In [5]:
print(mnist.train.next_batch(1)[0].shape)
print(mnist.train.next_batch(1)[1].shape)

(1, 784)
(1, 10)


##### it shows that mnist  triain shape is X=(55000,784) Y=(55000,10)

In [6]:
x=tf.placeholder(tf.float32,shape=[None,784])
y=tf.placeholder(tf.float32,shape=[None,10])

In [7]:
#lets initialize our weights and biases
hidden_1_layer={
    'weights':tf.Variable(tf.random_normal(shape=(784,n_nodes_hl1))),#(nx*nh)
    'biases':tf.Variable(tf.random_normal(shape=[n_nodes_hl1,1]))
}
hidden_2_layer={
    'weights':tf.Variable(tf.random_normal(shape=[n_nodes_hl1,n_nodes_hl2])),
    'biases':tf.Variable(tf.random_normal(shape=[n_nodes_hl2,1]))
}
hidden_3_layer={
    'weights':tf.Variable(tf.random_normal(shape=[n_nodes_hl2,n_nodes_hl3])),
    'biases':tf.Variable(tf.random_normal(shape=[n_nodes_hl3,1]))
}
output_layer={
    'weights':tf.Variable(tf.random_normal(shape=[n_nodes_hl3,n_classes])),
    'biases':tf.Variable(tf.random_normal(shape=[n_classes,1]))
}

In [8]:
l1=tf.add(tf.matmul(hidden_1_layer['weights'],x,transpose_a=True,transpose_b=True),hidden_1_layer['biases'])
a1=tf.nn.relu(l1)

l2=tf.add(tf.matmul(hidden_2_layer['weights'],a1,transpose_a=True),hidden_2_layer['biases'])
a2=tf.nn.relu(l2)

l3=tf.add(tf.matmul(hidden_3_layer['weights'],a2,transpose_a=True),hidden_3_layer['biases'])
a3=tf.nn.relu(l3)

output=tf.add(tf.matmul(output_layer['weights'],a3,transpose_a=True),output_layer['biases'])


In [21]:

lamb=0.005
regularizer = tf.nn.l2_loss(hidden_1_layer['weights']) + tf.nn.l2_loss(hidden_2_layer['weights']) + tf.nn.l2_loss(hidden_3_layer['weights'])

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=tf.transpose(output),labels=y)) 


cost = tf.reduce_mean(cost+lamb*regularizer)

optimizer=tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)


hm_epochs=12

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(hm_epochs):
        epoch_loss=0
        for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                
                _,c=sess.run([optimizer,cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c
                
                
        print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
        
        
       
    correct = tf.equal(tf.argmax(tf.transpose(output), 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    print('Accuracy on training data:',accuracy.eval({x:mnist.train.images, y:mnist.train.labels})) 
    
    correct = tf.equal(tf.argmax(tf.transpose(output), 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    print('Accuracy on test data:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))    

Epoch 0 completed out of 12 loss: 1414524.560546875
Epoch 1 completed out of 12 loss: 816666.6416015625
Epoch 2 completed out of 12 loss: 723436.0053710938
Epoch 3 completed out of 12 loss: 644313.1030273438
Epoch 4 completed out of 12 loss: 576812.3127441406
Epoch 5 completed out of 12 loss: 513709.61071777344
Epoch 6 completed out of 12 loss: 453199.42626953125
Epoch 7 completed out of 12 loss: 399709.45080566406
Epoch 8 completed out of 12 loss: 344495.8790283203
Epoch 9 completed out of 12 loss: 296518.5770263672
Epoch 10 completed out of 12 loss: 249116.41772460938
Epoch 11 completed out of 12 loss: 207514.44631958008
Accuracy on training data: 0.97949094
Accuracy on test data: 0.9564


###### Result: accuracy of 95.6% on test data and 97.9% on training data.